In [1]:
import warnings
warnings.simplefilter('ignore')

# imports

import pandas as pd
import numpy as np

from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import Normalizer

In [2]:
#read csv files in
df = pd.read_csv("Result/NBA_data_all_80-19.csv")
df2020 = pd.read_csv("Result/NBA_data_all_19-20.csv")

In [3]:
# scale stats by year
scaler = QuantileTransformer(output_distribution='normal')
normalizer = Normalizer()

norm_scaled_dfs = []

stay_columns = ["RK", "Team", "Year", "G", "W", "L", "Conf"]

a = df["Year"].min()
b = df["Year"].max()
x = b - a + 1
for i in range (x):
    year = a+i
    df_year = df.loc[df["Year"] == year]
    stay_df = df_year[stay_columns].reset_index(drop=True)   
    change_df = df_year.drop(columns=stay_columns).reset_index(drop=True)
    
#   scale data
    scaled_df = pd.DataFrame(scaler.fit_transform(change_df), columns=change_df.columns)
#   normalize data
    normalized_df = pd.DataFrame(normalizer.fit_transform(scaled_df), columns=scaled_df.columns)    
    
    merged_df = stay_df.merge(change_df, left_index=True, right_index=True)
    merged_df = merged_df.merge(normalized_df, left_index=True, right_index=True, suffixes = ('', '_scalnorm'))
    
    norm_scaled_dfs.append(merged_df)

final_df = pd.concat(norm_scaled_dfs, axis=0)
final_df = final_df.dropna()

# scale 2020 df
stay_df2020 = df2020[stay_columns].reset_index(drop=True)   
change_df2020 = df2020.drop(columns=stay_columns).reset_index(drop=True)

scaled_df2020 = pd.DataFrame(scaler.fit_transform(change_df2020), columns=change_df2020.columns)
normalized_df = pd.DataFrame(normalizer.fit_transform(scaled_df2020), columns=scaled_df2020.columns) 

merged_df2020 = stay_df2020.merge(change_df2020, left_index=True, right_index=True)
merged_df2020 = merged_df2020.merge(normalized_df, left_index=True, right_index=True, suffixes = ('', '_scalnorm'))
final_df2020 = merged_df2020 

In [4]:
# creating new columns
final_df["playoffs_y_n"] = final_df["Team"].map(lambda x: 1 if "*" in x else 0)
final_df["Team"] = final_df["Team"].str.replace("*", "")
final_df["W_%"] = final_df["W"]/final_df["G"]

final_df2020["playoffs_y_n"] = final_df2020["Team"].map(lambda x: 1 if "*" in x else 0)
final_df2020["Team"] = final_df2020["Team"].str.replace("*", "")
final_df2020["W_%"] = final_df2020["W"]/final_df2020["G"]

In [5]:
final_df2020.to_csv("NBA_data/preprocessed_2020.csv")
final_df.to_csv("NBA_data/preprocessed_80-19.csv")